## Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

# Part 3

In [2]:
import pandas as pd
import numpy as np
import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.21.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

### Get the dataframe from first task

In [3]:
# read data from html
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
read_table = pd.read_html(url,header=[0])
df1 = read_table[0]

# rename columns' name
df1 = df1.rename(index=str, columns={'Postcode':'PostalCode','Neighbourhood':'Neighborhood'})

# Ignore cells with a borough that is Not assigned
df1 = df1[df1.Borough !='Not assigned']
df1.reset_index(drop=True,inplace=True)

# groupby
df1 = df1.groupby('PostalCode',as_index=False).agg(lambda x: ','.join(set(x.dropna())))

# If a cell has a borough but a Not assigned neighborhood, 
#then the neighborhood will be the same as the borough
df1.loc[df1['Neighborhood'] == 'Not assigned','Neighborhood'] = df1['Borough']

df1.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Rouge Hill,Highland Creek,Port Union"
2,M1E,Scarborough,"West Hill,Morningside,Guildwood"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Get the dataframe from Second task

In [4]:
df2 = pd.read_csv('http://cocl.us/Geospatial_data')

# Change the columns' name
df2.columns = ['PostalCode','Latitude','Longitude']

# Merge two dataframes
df = pd.merge(left=df1,right=df2,on='PostalCode')
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill,Highland Creek,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"West Hill,Morningside,Guildwood",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Part 3 

### Get Toronto data

In [5]:
Toronto = df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)
Toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


### Get the latitude and longitude values of Toronto

In [6]:
address = 'Toronto'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are{},{}.'.format(latitude,longitude))

The geograpical coordinate of Toronto are43.653963,-79.387207.


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


### Create map of Toronto

In [7]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Toronto['Latitude'], Toronto['Longitude'], Toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

### Define Foursquare Credentials and Version

In [8]:
CLIENT_ID = 'HCIWEYMLE0SJAI3ESV4AFX5PNQVBSLP5HQ1YU4GISAHHRIFV' # your Foursquare ID
CLIENT_SECRET = 'P4KVBEVJDIVREULUPIZHUL124JX353PUIP5KWJOGX1PLDB5B' # your Foursquare Secret
VERSION = '20200202' # Foursquare API version

### Try to explore other neighborhood
#### pick up Etobicoke to be explored

In [9]:
Etobicoke = df[df['Borough']=='Etobicoke'].reset_index(drop=True)

# get the Etobicoke latitude and longitude values
Etobicoke_latitude = Etobicoke.loc[0,'Latitude']
Etobicoke_longitude = Etobicoke.loc[0,'Longitude']

print('Latitude and longitude values of Etobicoke are {},{}.'.format(
    Etobicoke_latitude, Etobicoke_longitude))

Latitude and longitude values of Etobicoke are 43.6056466,-79.50132070000001.


In [10]:
map_Etobicoke = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Etobicoke['Latitude'], Etobicoke['Longitude'], Etobicoke['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Etobicoke)  
    
map_Etobicoke

### Explore Neighborhoods in Toronto
#### Getting the top 100 venues that are in Toronto within a raidus of 500 meters


In [11]:
LIMIT = 100
radius = 500

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


# write the code to run the above function
Toronto_venues = getNearbyVenues(names=Toronto['Neighborhood'],
                                  latitudes=Toronto['Latitude'],
                                  longitudes=Toronto['Longitude'])


The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Rathnelly,Summerhill West,South Hill,Deer Park,Forest Hill SE
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront
Garden District,Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond,Adelaide,King
Harbourfront East,Union Station,Toronto Islands
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill West,Forest Hill North
Yorkville,North Midtown,The Annex
University of Toronto,Harbord
Chinatown,Kensington Market,Grange Park
South Niagara,CN Tower,Island airport,Bathurst Quay,Harbourfront West,Railway Lands,King and Spadina
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Trinity,Little Portugal
Parkdale Village,Brockton,Exhibition Place
High Park,The Junction South
Roncesvalles,Parkdale
Runnymede

### Check the size of the resulting dataframe

In [12]:
print(Toronto_venues.shape)
Toronto_venues.head()

(1706, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West,Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


### Check how many venues were returned

In [13]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,55,55,55,55,55,55
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"Cabbagetown,St. James Town",42,42,42,42,42,42
Central Bay Street,84,84,84,84,84,84
"Chinatown,Kensington Market,Grange Park",80,80,80,80,80,80
Christie,18,18,18,18,18,18
Church and Wellesley,81,81,81,81,81,81
"Commerce Court,Victoria Hotel",100,100,100,100,100,100
Davisville,35,35,35,35,35,35


### Find out how many unique categories can be curated from all the returned venues


In [14]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 229 uniques categories.


### Analyze Neighborhood

In [15]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### New dataframe shape

In [16]:
Toronto_onehot.shape

(1706, 229)

### Group rows by neigoborhood and by taking the mean of the frequency of occurrence of each category

In [17]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,Berczy Park,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.018182,0.000000,0.000000,0.000000,0.00
1,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
2,"Cabbagetown,St. James Town",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
3,Central Bay Street,0.011905,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.011905,...,0.000000,0.000000,0.000000,0.00000,0.00,0.011905,0.000000,0.000000,0.011905,0.00
4,"Chinatown,Kensington Market,Grange Park",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.050000,0.000000,0.050000,0.012500,0.00
5,Christie,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
6,Church and Wellesley,0.012346,0.012346,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.012346,...,0.012346,0.012346,0.000000,0.00000,0.00,0.000000,0.000000,0.012346,0.000000,0.00
7,"Commerce Court,Victoria Hotel",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.030000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.00
8,Davisville,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.028571,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
9,Davisville North,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00


### New size after grouping

In [18]:
# new size
Toronto_grouped.shape

(39, 229)

### Pick up top 10 venues

In [19]:
num_top_venues = 10

# write a funcition to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)   
    return row_categories_sorted.index.values[0:num_top_venues]

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Toronto_venues_sorted = pd.DataFrame(columns=columns)
Toronto_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    Toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

Toronto_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Seafood Restaurant,Steakhouse,Café,Cheese Shop,Beer Bar,Bakery,Breakfast Spot
1,Business Reply Mail Processing Centre 969 Eastern,Park,Burrito Place,Butcher,Fast Food Restaurant,Farmers Market,Auto Workshop,Recording Studio,Spa,Restaurant,Pizza Place
2,"Cabbagetown,St. James Town",Restaurant,Coffee Shop,Bakery,Café,Pizza Place,Italian Restaurant,Pub,Pharmacy,Playground,Snack Place
3,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Ice Cream Shop,Juice Bar,Burger Joint,Japanese Restaurant,Bar,Salad Place
4,"Chinatown,Kensington Market,Grange Park",Bar,Café,Coffee Shop,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Chinese Restaurant,Dumpling Restaurant,Mexican Restaurant,Donut Shop,Cocktail Bar
5,Christie,Grocery Store,Café,Park,Candy Store,Baby Store,Coffee Shop,Gas Station,Nightclub,Restaurant,Diner
6,Church and Wellesley,Coffee Shop,Gay Bar,Japanese Restaurant,Sushi Restaurant,Restaurant,Men's Store,Pub,Gastropub,Fast Food Restaurant,Gym
7,"Commerce Court,Victoria Hotel",Coffee Shop,Café,Hotel,Restaurant,Deli / Bodega,Gym,Bakery,Seafood Restaurant,Steakhouse,Gastropub
8,Davisville,Dessert Shop,Sandwich Place,Gym,Coffee Shop,Sushi Restaurant,Pizza Place,Café,Italian Restaurant,Gourmet Shop,Indian Restaurant
9,Davisville North,Hotel,Park,Gym,Breakfast Spot,Sandwich Place,Food & Drink Shop,Department Store,Women's Store,Donut Shop,Doner Restaurant


### Cluster Neighborhoods
Run k-means to cluster the neighborhood into 5 clusters.

In [20]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [21]:
Toronto_merged = Toronto

# add clustering labels
Toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(Toronto_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Health Food Store,Pub,Women's Store,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Ice Cream Shop,Furniture / Home Store,Yoga Studio,Pub,Bakery,Dessert Shop
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0,Park,Sushi Restaurant,Board Shop,Brewery,Italian Restaurant,Liquor Store,Burger Joint,Burrito Place,Ice Cream Shop,Fast Food Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Gastropub,Bakery,Brewery,Italian Restaurant,American Restaurant,Comfort Food Restaurant,Sandwich Place,Cheese Shop
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Bus Line,Lake,Swim School,Park,Colombian Restaurant,Comfort Food Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


### Visualize the resulting clusters

In [22]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], 
                                  Toronto_merged['Neighborhood'], 
                                  Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters